Lambda School Data Science, Unit 2: Predictive Modeling

# Kaggle Challenge, Module 3


## Assignment
- [ ] [Review requirements for your portfolio project](https://lambdaschool.github.io/ds/unit2), then submit your dataset.
- [ ] Continue to participate in our Kaggle challenge. 
- [ ] Use scikit-learn for hyperparameter optimization with RandomizedSearchCV.
- [ ] Submit your predictions to our Kaggle competition. (Go to our Kaggle InClass competition webpage. Use the blue **Submit Predictions** button to upload your CSV file. Or you can use the Kaggle API to submit your predictions.)
- [ ] Commit your notebook to your fork of the GitHub repo.

## Stretch Goals

### Reading
- Jake VanderPlas, [Python Data Science Handbook, Chapter 5.3](https://jakevdp.github.io/PythonDataScienceHandbook/05.03-hyperparameters-and-model-validation.html), Hyperparameters and Model Validation
- Jake VanderPlas, [Statistics for Hackers](https://speakerdeck.com/jakevdp/statistics-for-hackers?slide=107)
- Ron Zacharski, [A Programmer's Guide to Data Mining, Chapter 5](http://guidetodatamining.com/chapter5/), 10-fold cross validation
- Sebastian Raschka, [A Basic Pipeline and Grid Search Setup](https://github.com/rasbt/python-machine-learning-book/blob/master/code/bonus/svm_iris_pipeline_and_gridsearch.ipynb)
- Peter Worcester, [A Comparison of Grid Search and Randomized Search Using Scikit Learn](https://blog.usejournal.com/a-comparison-of-grid-search-and-randomized-search-using-scikit-learn-29823179bc85)

### Doing
- In additon to `RandomizedSearchCV`, scikit-learn has [`GridSearchCV`](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html). Another library called scikit-optimize has [`BayesSearchCV`](https://scikit-optimize.github.io/notebooks/sklearn-gridsearchcv-replacement.html). Experiment with these alternatives.
- _[Introduction to Machine Learning with Python](http://shop.oreilly.com/product/0636920030515.do)_ discusses options for "Grid-Searching Which Model To Use" in Chapter 6:

> You can even go further in combining GridSearchCV and Pipeline: it is also possible to search over the actual steps being performed in the pipeline (say whether to use StandardScaler or MinMaxScaler). This leads to an even bigger search space and should be considered carefully. Trying all possible solutions is usually not a viable machine learning strategy. However, here is an example comparing a RandomForestClassifier and an SVC ...

The example is shown in [the accompanying notebook](https://github.com/amueller/introduction_to_ml_with_python/blob/master/06-algorithm-chains-and-pipelines.ipynb), code cells 35-37. Could you apply this concept to your own pipelines?


### More Categorical Encodings

**1.** The article **[Categorical Features and Encoding in Decision Trees](https://medium.com/data-design/visiting-categorical-features-and-encoding-in-decision-trees-53400fa65931)** mentions 4 encodings:

- **"Categorical Encoding":** This means using the raw categorical values as-is, not encoded. Scikit-learn doesn't support this, but some tree algorithm implementations do. For example, [Catboost](https://catboost.ai/), or R's [rpart](https://cran.r-project.org/web/packages/rpart/index.html) package.
- **Numeric Encoding:** Synonymous with Label Encoding, or "Ordinal" Encoding with random order. We can use [category_encoders.OrdinalEncoder](https://contrib.scikit-learn.org/categorical-encoding/ordinal.html).
- **One-Hot Encoding:** We can use [category_encoders.OneHotEncoder](http://contrib.scikit-learn.org/categorical-encoding/onehot.html).
- **Binary Encoding:** We can use [category_encoders.BinaryEncoder](http://contrib.scikit-learn.org/categorical-encoding/binary.html).


**2.** The short video 
**[Coursera — How to Win a Data Science Competition: Learn from Top Kagglers — Concept of mean encoding](https://www.coursera.org/lecture/competitive-data-science/concept-of-mean-encoding-b5Gxv)** introduces an interesting idea: use both X _and_ y to encode categoricals.

Category Encoders has multiple implementations of this general concept:

- [CatBoost Encoder](http://contrib.scikit-learn.org/categorical-encoding/catboost.html)
- [James-Stein Encoder](http://contrib.scikit-learn.org/categorical-encoding/jamesstein.html)
- [Leave One Out](http://contrib.scikit-learn.org/categorical-encoding/leaveoneout.html)
- [M-estimate](http://contrib.scikit-learn.org/categorical-encoding/mestimate.html)
- [Target Encoder](http://contrib.scikit-learn.org/categorical-encoding/targetencoder.html)
- [Weight of Evidence](http://contrib.scikit-learn.org/categorical-encoding/woe.html)

Category Encoder's mean encoding implementations work for regression problems or binary classification problems. 

For multi-class classification problems, you will need to temporarily reformulate it as binary classification. For example:

```python
encoder = ce.TargetEncoder(min_samples_leaf=..., smoothing=...) # Both parameters > 1 to avoid overfitting
X_train_encoded = encoder.fit_transform(X_train, y_train=='functional')
X_val_encoded = encoder.transform(X_train, y_val=='functional')
```

**3.** The **[dirty_cat](https://dirty-cat.github.io/stable/)** library has a Target Encoder implementation that works with multi-class classification.

```python
 dirty_cat.TargetEncoder(clf_type='multiclass-clf')
```
It also implements an interesting idea called ["Similarity Encoder" for dirty categories](https://www.slideshare.net/GaelVaroquaux/machine-learning-on-non-curated-data-154905090).

However, it seems like dirty_cat doesn't handle missing values or unknown categories as well as category_encoders does. And you may need to use it with one column at a time, instead of with your whole dataframe.

**4. [Embeddings](https://www.kaggle.com/learn/embeddings)** can work well with sparse / high cardinality categoricals.

_**I hope it’s not too frustrating or confusing that there’s not one “canonical” way to encode categorcals. It’s an active area of research and experimentation! Maybe you can make your own contributions!**_

### BONUS: Stacking!

Here's some code you can use to "stack" multiple submissions, which is another form of ensembling:

```python
import pandas as pd

# Filenames of your submissions you want to ensemble
files = ['submission-01.csv', 'submission-02.csv', 'submission-03.csv']

target = 'status_group'
submissions = (pd.read_csv(file)[[target]] for file in files)
ensemble = pd.concat(submissions, axis='columns')
majority_vote = ensemble.mode(axis='columns')[0]

sample_submission = pd.read_csv('sample_submission.csv')
submission = sample_submission.copy()
submission[target] = majority_vote
submission.to_csv('my-ultimate-ensemble-submission.csv', index=False)
```

In [1]:
import os, sys
in_colab = 'google.colab' in sys.modules

# If you're in Colab...
if in_colab:
    # Pull files from Github repo
    os.chdir('/content')
    !git init .
    !git remote add origin https://github.com/LambdaSchool/DS-Unit-2-Kaggle-Challenge.git
    !git pull origin master
    
    # Install required python packages
    !pip install -r requirements.txt
    
    # Change into directory for module
    os.chdir('module3')

In [2]:
import pandas as pd

# Merge train_features.csv & train_labels.csv
train = pd.merge(pd.read_csv('../data/tanzania/train_features.csv'), 
                 pd.read_csv('../data/tanzania/train_labels.csv'))

# Read test_features.csv & sample_submission.csv
test = pd.read_csv('../data/tanzania/test_features.csv')
sample_submission = pd.read_csv('../data/tanzania/sample_submission.csv')

In [3]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
%matplotlib inline

train, val = train_test_split(train, train_size = 0.8, test_size = 0.2, random_state=42)
print(train.shape)
print(val.shape)
test.shape

def wrangle(X):
    '''Wrangle train, validate and test sets'''
    
    # this will prevent SettingWithCopyWarning
    X = X.copy()
    
    # latitude and longitude have some missing values coded as 0s (or very small near-zeros.)
    # I'm going to replace those with nulls for now and later decide how to impute them
    X['latitude'] = X['latitude'].replace(-2e-08, 0)
    
    cols_with_zeros = ['latitude', 'longitude', 'construction_year', 'gps_height',
                      ]
    for col in cols_with_zeros:
        X[col] = X[col].replace(0, np.nan)
        

    # quantity and quantity_group are the same data, so we don't need both    
    # recorded_by only has one value, which gives us literally no variance
    # num_private doesnt seem to help the model
    # wpt_name has very high cardinality
    # extraction_type_group and extraction_type are both similar to extraction_type_class
    # payment_type is similar to payment
    # water_quality is similar to quality_group
    # basin and scheme_management don't help the model
    # waterpoint_type_gruop is similar to waterpoint_type
    # quantity_group is a duplicate of quantity
    # installer has very high cardinality
    X = X.drop(columns=['quantity_group', 'recorded_by', 'num_private', 'wpt_name',
              'extraction_type_group', 'extraction_type', 'payment_type', 'water_quality',
              'basin', 'scheme_management', 'waterpoint_type_group', 'installer'])
    
    return X

train = wrangle(train)
val = wrangle(val)
test = wrangle(test)

(47520, 41)
(11880, 41)


In [23]:
import category_encoders as ce
from sklearn.impute import SimpleImputer
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import make_pipeline
from sklearn.feature_selection import SelectKBest
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import StandardScaler

target = 'status_group'

X_train = train.drop(columns=target)
y_train = train[target]
X_val = val.drop(columns=target)
y_val = val[target]
X_test = test

pipeline = make_pipeline(
#     ce.TargetEncoder(min_samples_leaf=1, smoothing=1),
    ce.OrdinalEncoder(),
    SimpleImputer(strategy='median'),
    RandomForestClassifier(n_estimators=30, random_state=42, n_jobs=-1, max_depth=20, min_samples_split=5, 
                          min_samples_leaf=1)
)

k = 3
scores = cross_val_score(pipeline, X_train, y_train, cv=k,
                        scoring='accuracy')
print(f'MAE for {k} folds', scores)


MAE for 3 folds [0.79609873 0.80018939 0.79689374]


In [24]:
print(scores.mean())
scores.std()


0.7977272894573265


0.0017709658269970789

In [25]:
print('Model Hyperparameters:')
print(pipeline.named_steps['randomforestclassifier'])

Model Hyperparameters:
RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=20, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=5,
                       min_weight_fraction_leaf=0.0, n_estimators=30, n_jobs=-1,
                       oob_score=False, random_state=42, verbose=0,
                       warm_start=False)


In [ ]:
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from scipy.stats import randint, uniform

pipeline = make_pipeline(
    ce.OrdinalEncoder(),
    SimpleImputer(strategy='median'),
    StandardScaler(),
    RandomForestClassifier(random_state=42, n_jobs=-1)
)

param_distributions = {
    'simpleimputer__strategy':['mean', 'median'],
    'randomforestclassifier__n_estimators': randint(20, 500),
    'randomforestclassifier__max_depth': [5, 10, 15, 20, None],
    'randomforestclassifier__max_features': uniform(0, 1),
    'randomforestclassifier__min_samples_split': range(2, 10),
    'randomforestclassifier__min_samples_leaf': range(1, 10)
}

search = RandomizedSearchCV(
    pipeline,
    param_distributions=param_distributions,
    n_iter=100,
    cv=10,
    scoring='accuracy',
    verbose=10,
    return_train_score=True,
#     n_jobs=-1
)

search.fit(X_train, y_train)

Fitting 10 folds for each of 100 candidates, totalling 1000 fits
[CV] randomforestclassifier__max_depth=10, randomforestclassifier__max_features=0.17366068626533937, randomforestclassifier__min_samples_leaf=4, randomforestclassifier__min_samples_split=9, randomforestclassifier__n_estimators=463, simpleimputer__strategy=mean 


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


[CV]  randomforestclassifier__max_depth=10, randomforestclassifier__max_features=0.17366068626533937, randomforestclassifier__min_samples_leaf=4, randomforestclassifier__min_samples_split=9, randomforestclassifier__n_estimators=463, simpleimputer__strategy=mean, score=(train=0.772, test=0.752), total=   9.0s
[CV] randomforestclassifier__max_depth=10, randomforestclassifier__max_features=0.17366068626533937, randomforestclassifier__min_samples_leaf=4, randomforestclassifier__min_samples_split=9, randomforestclassifier__n_estimators=463, simpleimputer__strategy=mean 


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    9.9s remaining:    0.0s


[CV]  randomforestclassifier__max_depth=10, randomforestclassifier__max_features=0.17366068626533937, randomforestclassifier__min_samples_leaf=4, randomforestclassifier__min_samples_split=9, randomforestclassifier__n_estimators=463, simpleimputer__strategy=mean, score=(train=0.779, test=0.771), total=   8.6s
[CV] randomforestclassifier__max_depth=10, randomforestclassifier__max_features=0.17366068626533937, randomforestclassifier__min_samples_leaf=4, randomforestclassifier__min_samples_split=9, randomforestclassifier__n_estimators=463, simpleimputer__strategy=mean 


...

[CV]  randomforestclassifier__max_depth=20, randomforestclassifier__max_features=0.8774962645677027, randomforestclassifier__min_samples_leaf=9, randomforestclassifier__min_samples_split=7, randomforestclassifier__n_estimators=287, simpleimputer__strategy=median, score=(train=0.860, test=0.799), total=  26.3s
[CV] randomforestclassifier__max_depth=20, randomforestclassifier__max_features=0.8774962645677027, randomforestclassifier__min_samples_leaf=9, randomforestclassifier__min_samples_split=7, randomforestclassifier__n_estimators=287, simpleimputer__strategy=median 
[CV]  randomforestclassifier__max_depth=20, randomforestclassifier__max_features=0.8774962645677027, randomforestclassifier__min_samples_leaf=9, randomforestclassifier__min_samples_split=7, randomforestclassifier__n_estimators=287, simpleimputer__strategy=median, score=(train=0.862, test=0.794), total=  26.8s
[CV] randomforestclassifier__max_depth=20, randomforestclassifier__max_features=0.8774962645677027, randomforestcla

[CV]  randomforestclassifier__max_depth=20, randomforestclassifier__max_features=0.20799792567827446, randomforestclassifier__min_samples_leaf=6, randomforestclassifier__min_samples_split=3, randomforestclassifier__n_estimators=300, simpleimputer__strategy=mean, score=(train=0.863, test=0.796), total=   8.0s
[CV] randomforestclassifier__max_depth=15, randomforestclassifier__max_features=0.15475547343192353, randomforestclassifier__min_samples_leaf=6, randomforestclassifier__min_samples_split=6, randomforestclassifier__n_estimators=116, simpleimputer__strategy=median 
[CV]  randomforestclassifier__max_depth=15, randomforestclassifier__max_features=0.15475547343192353, randomforestclassifier__min_samples_leaf=6, randomforestclassifier__min_samples_split=6, randomforestclassifier__n_estimators=116, simpleimputer__strategy=median, score=(train=0.831, test=0.789), total=   3.2s
[CV] randomforestclassifier__max_depth=15, randomforestclassifier__max_features=0.15475547343192353, randomforestc

[CV]  randomforestclassifier__max_depth=20, randomforestclassifier__max_features=0.8783963148052109, randomforestclassifier__min_samples_leaf=1, randomforestclassifier__min_samples_split=8, randomforestclassifier__n_estimators=481, simpleimputer__strategy=mean, score=(train=0.935, test=0.805), total=  51.1s
[CV] randomforestclassifier__max_depth=20, randomforestclassifier__max_features=0.8783963148052109, randomforestclassifier__min_samples_leaf=1, randomforestclassifier__min_samples_split=8, randomforestclassifier__n_estimators=481, simpleimputer__strategy=mean 
[CV]  randomforestclassifier__max_depth=20, randomforestclassifier__max_features=0.8783963148052109, randomforestclassifier__min_samples_leaf=1, randomforestclassifier__min_samples_split=8, randomforestclassifier__n_estimators=481, simpleimputer__strategy=mean, score=(train=0.935, test=0.799), total=  51.6s
[CV] randomforestclassifier__max_depth=20, randomforestclassifier__max_features=0.8783963148052109, randomforestclassifie

[CV]  randomforestclassifier__max_depth=20, randomforestclassifier__max_features=0.6711874995260162, randomforestclassifier__min_samples_leaf=4, randomforestclassifier__min_samples_split=5, randomforestclassifier__n_estimators=55, simpleimputer__strategy=mean, score=(train=0.904, test=0.801), total=   4.8s
[CV] randomforestclassifier__max_depth=20, randomforestclassifier__max_features=0.4030994078320471, randomforestclassifier__min_samples_leaf=4, randomforestclassifier__min_samples_split=7, randomforestclassifier__n_estimators=373, simpleimputer__strategy=median 
[CV]  randomforestclassifier__max_depth=20, randomforestclassifier__max_features=0.4030994078320471, randomforestclassifier__min_samples_leaf=4, randomforestclassifier__min_samples_split=7, randomforestclassifier__n_estimators=373, simpleimputer__strategy=median, score=(train=0.897, test=0.801), total=  19.4s
[CV] randomforestclassifier__max_depth=20, randomforestclassifier__max_features=0.4030994078320471, randomforestclassi

[CV]  randomforestclassifier__max_depth=None, randomforestclassifier__max_features=0.09578539041163403, randomforestclassifier__min_samples_leaf=5, randomforestclassifier__min_samples_split=5, randomforestclassifier__n_estimators=48, simpleimputer__strategy=median, score=(train=0.852, test=0.790), total=   1.6s
[CV] randomforestclassifier__max_depth=None, randomforestclassifier__max_features=0.09578539041163403, randomforestclassifier__min_samples_leaf=5, randomforestclassifier__min_samples_split=5, randomforestclassifier__n_estimators=48, simpleimputer__strategy=median 
[CV]  randomforestclassifier__max_depth=None, randomforestclassifier__max_features=0.09578539041163403, randomforestclassifier__min_samples_leaf=5, randomforestclassifier__min_samples_split=5, randomforestclassifier__n_estimators=48, simpleimputer__strategy=median, score=(train=0.851, test=0.789), total=   1.4s
[CV] randomforestclassifier__max_depth=None, randomforestclassifier__max_features=0.09578539041163403, random

In [ ]:
print('Best hyperparameters', search.best_params_)
print('Cross-validation accuracy', search.best_score_)

In [ ]:
pipeline = search.best_estimator_

In [ ]:
pipeline.fit(X_train, y_train)
print('Validation accuracy:', pipeline.score(X_val, y_val))

In [ ]:
rf = pipeline.named_steps['randomforestclassifier']
encoder = pipeline.named_steps['ordinalencoder']
encoded = encoder.transform(X_train)

importances = pd.Series(rf.feature_importances_, encoded.columns)

n = 20
plt.figure(figsize=(10, 10))
importances.sort_values()[-n:].plot.barh(color='b');

In [ ]:
y_pred = pipeline.predict(X_test)
submission = sample_submission.copy()
submission['status_group'] = y_pred
submission.to_csv('submission-01.csv', index=False)